In [1]:
import twint
import pandas as pd
import nest_asyncio
import time
nest_asyncio.apply()
pd.options.display.max_columns = 40


In [2]:
# candidate = 'Biden'

# c = twint.Config()
# c.Search = candidate
# c.Lang = 'en'
# c.Since = '2020-10-01'
# c.Until = '2020-10-05'
# c.Store_csv = True
# c.Output = '../data/biden_tweets_all.csv'
# c.Hide_output = True
# twint.run.Search(c)

## Scrape Tweets

This is the workbook I used to create the below function.

Note...this takes a while. In order to prevent Twint's `client payload error`, I had to add randomized wait times between each Twitter query. I suggest running the below cells overnight to generate the files.

In [5]:
import datetime as dt
import time
import random

def arrange_dates(start_date, end_date):
    """Used in the get_election_tweets func. """
    
    dates = []
    delta = end_date - start_date
    
    for i in range(delta.days + 1):
        day = start_date + dt.timedelta(days=i)
        dates.append(day)
    
    return dates

def get_election_tweets(tweets_per_day=15000, start_date="2020-10-01", end_date="2020-11-03"):
    
    # get a list of dates
    f = "%Y-%m-%d"
    sd = dt.datetime.strptime(start_date, f)
    ed = dt.datetime.strptime(end_date, f)
    dates = arrange_dates(sd, ed)
    candidates = ['Biden', 'Trump']
    
    # fetch tweets & save CSVs (each CSV contains one day/candidate)
    
    lam_plus_1 = lambda x: (x + dt.timedelta(days=1)).strftime("%Y-%m-%d")
    wait_time = 60 # seconds; will use for sleep timer to avoid 'client payload' error
    print("Waiting 60 seconds...")
    time.sleep(60)
    for date in dates:
        for candidate in candidates:
            
            d = date.strftime("%Y-%m-%d")            
            print(f"""\n Begin scraping {candidate}'s tweets on {d}\n \n""")
            outfile = f"../data/tweets/{candidate}_{d}.csv"
            
            # twint search query details
            c = twint.Config()
            c.Search = candidate
            c.Lang = 'en'
            c.Since = d
            c.Until = lam_plus_1(date)
            c.Limit = tweets_per_day
            c.Store_csv = True
            c.Output = outfile
            c.Hide_output = True
            
            # run the search
            try:
                twint.run.Search(c)
                print(f"Scraping complete. File: {outfile} created.")
            except:
                time.sleep(90)
                continue
            
            # wait some time so script does not receive 'client payload' error
            print(f"Waiting 140-295 seconds...")
            time.sleep(wait_time + random.choice([140, 240, 290]) + (5 * random.uniform(0, 1)))
                       

In [6]:
get_election_tweets(start_date="2020-10-01", end_date="2020-11-02")

Waiting 60 seconds...

 Begin scraping Biden's tweets on 2020-10-01
 

Scraping complete. File: ../data/tweets/Biden_2020-10-01.csv created.
Waiting 60-100 seconds...

 Begin scraping Trump's tweets on 2020-10-01
 



ClientPayloadError: Response payload is not completed